# --- Day 19: Linen Layout ---

Every towel at this onsen is marked with a pattern of colored stripes. There are only a few patterns, but for any particular pattern, the staff can get you as many towels with that pattern as you need. Each stripe can be white (w), blue (u), black (b), red (r), or green (g). So, a towel with the pattern ggr would have a green stripe, a green stripe, and then a red stripe, in that order. (You can't reverse a pattern by flipping a towel upside-down, as that would cause the onsen logo to face the wrong way.)

The Official Onsen Branding Expert has produced a list of designs - each a long sequence of stripe colors - that they would like to be able to display. You can use any towels you want, but all of the towels' stripes must exactly match the desired design. So, to display the design rgrgr, you could use two rg towels and then an r towel, an rgr towel and then a gr towel, or even a single massive rgrgr towel (assuming such towel patterns were actually available).

To start, collect together all of the available towel patterns and the list of desired designs (your puzzle input). For example:
```
r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb
```
The first line indicates the available towel patterns; in this example, the onsen has unlimited towels with a single red stripe (r), unlimited towels with a white stripe and then a red stripe (wr), and so on.

After the blank line, the remaining lines each describe a design the onsen would like to be able to display. In this example, the first design (brwrr) indicates that the onsen would like to be able to display a black stripe, a red stripe, a white stripe, and then two red stripes, in that order.

Not all designs will be possible with the available towels. In the above example, the designs are possible or impossible as follows:

- brwrr can be made with a br towel, then a wr towel, and then finally an r towel.
- bggr can be made with a b towel, two g towels, and then an r towel.
- gbbr can be made with a gb towel and then a br towel.
- rrbgbr can be made with r, rb, g, and br.
- ubwu is impossible.
- bwurrg can be made with bwu, r, r, and g.
- brgr can be made with br, g, and r.
- bbrgwb is impossible.

In this example, 6 of the eight designs are possible with the available towel patterns.

To get into the onsen as soon as possible, consult your list of towel patterns and desired designs carefully. How many designs are possible?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=19, year=2024)

samp = """r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb"""
print(samp)

r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb


Summarize the task for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠

You have a set of available towel patterns (like "r", "wr", "gb", etc.) and a list of target designs. Your goal is to determine how many designs can be constructed by concatenating the available towel patterns (each pattern can be used unlimited times).

Essentially, it's a string matching problem: can you build each target string by combining the given substrings?

I'm thinking how to approach this problem. I could simply go left to right and check all the possible combinations. But I wonder if there is a better approach.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Your left-to-right idea is solid! The key insight is that if you try every combination naively, you might repeat a lot of work. For example, checking if "rgrgr" is possible might involve checking if "grgr" is possible multiple times via different paths.

Have you considered using **memoization** or **dynamic programming** to avoid recalculating the same subproblems? What would you cache?

Let's explore the memoization. I was thinking about a recursive algorithm where I'd remove a prefix and check if the substring can be assembled as a subproblem.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! That's the classic approach here. You'd:

1. Check if any towel pattern matches the start of your design
2. If so, recursively check if the *remainder* can be built
3. Cache results for each suffix you've already checked

What would your base case be?

I'm still not sure how the caching will look like. I guess the function will take a string to be checked and a list of patterns as an input and it will return a boolean if it can or cannot be assembled. Is that correct?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, that's correct! The function signature would be something like `can_build(s, patterns) -> bool`.

For caching, since `patterns` stays constant across all calls, you only need to cache based on the string `s` (or equivalently, the starting index into the original design). So your cache maps each suffix you've tried → whether it's buildable.

Want to try writing the recursive function first without memoization, then we can add caching?

Here's the plan:
1. Write `can_build(s, index, patterns) -> bool`. This way, the cache key is just an integer instead of a string.
2. Add caching.

Does the plan look sound?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, the plan looks sound! Using the index as the cache key is efficient. Go ahead and try implementing step 1 — the recursive function without caching first.

In [ ]:
"brwrr".startswith("br")

True

In [ ]:
ps_str, ds_str = samp.split('\n\n')
ps_str, ds_str

('r, wr, b, g, bwu, rb, gb, br',
 'brwrr\nbggr\ngbbr\nrrbgbr\nubwu\nbwurrg\nbrgr\nbbrgwb')

In [ ]:
ps = ps_str.split(', ')
ps

['r', 'wr', 'b', 'g', 'bwu', 'rb', 'gb', 'br']

In [ ]:
ds = ds_str.splitlines()
ds

['brwrr', 'bggr', 'gbbr', 'rrbgbr', 'ubwu', 'bwurrg', 'brgr', 'bbrgwb']

In [ ]:
d = ds[0]
d

'brwrr'

In [ ]:
def parse(inp):
    ps_str, ds_str = inp.split('\n\n')
    ps = ps_str.split(', ')
    ds = ds_str.splitlines()
    return L(ps), L(ds)
ps, ds = parse(samp)
ps, ds

(['r', 'wr', 'b', 'g', 'bwu', 'rb', 'gb', 'br'],
 ['brwrr', 'bggr', 'gbbr', 'rrbgbr', 'ubwu', 'bwurrg', 'brgr', 'bbrgwb'])

In [ ]:
def can_build(s, index, patterns):
    if index == len(s):
        return True
    return any(s[index:].startswith(p) and can_build(s, index + len(p), patterns) for p in patterns)

In [ ]:
L(ds).map(can_build, index=0, patterns=ps)

[True, True, True, True, False, True, True, False]

How can I add memoization?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The simplest way is Python's `@functools.cache` decorator. But there's a catch — `patterns` is a list, which isn't hashable.

Two options:
1. Move `patterns` outside the function (or use a closure) so only `index` is a parameter
2. Convert `patterns` to a tuple when passing it

Which approach would you like to try?

In [ ]:
import functools

def can_build(s, patterns):
    @functools.cache
    def can_build_rec(s, index):
        if index == len(s):
            return True
        return any(s[index:].startswith(p) and can_build_rec(s, index + len(p)) for p in patterns)
    return can_build_rec(s, 0)
L(ds).map(can_build, patterns=ps)

[True, True, True, True, False, True, True, False]

In [ ]:
def solve(inp):
    ps, ds = parse(inp)
    return ds.map(can_build, patterns=ps).sum()
solve(samp)

6

In [ ]:
solve(inp)

209

# --- Part Two ---
The staff don't really like some of the towel arrangements you came up with. To avoid an endless cycle of towel rearrangement, maybe you should just give them every possible option.

Here are all of the different ways the above example's designs can be made:

brwrr can be made in two different ways: b, r, wr, r or br, wr, r.

bggr can only be made with b, g, g, and r.

gbbr can be made 4 different ways:

- g, b, b, r
- g, b, br
- gb, b, r
- gb, br

rrbgbr can be made 6 different ways:

- r, r, b, g, b, r
- r, r, b, g, br
- r, r, b, gb, r
- r, rb, g, b, r
- r, rb, g, br
- r, rb, gb, r

bwurrg can only be made with bwu, r, r, and g.

brgr can be made in two different ways: b, r, g, r or br, g, r.

ubwu and bbrgwb are still impossible.

Adding up all of the ways the towels in this example could be arranged into the desired designs yields 16 (2 + 1 + 4 + 6 + 1 + 2).

They'll let you into the onsen as soon as you have the list. What do you get if you add up the number of different ways you could make each design?

Summarize the part 2 for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠

Instead of checking *if* a design can be built, you now need to count *how many ways* each design can be built. Then sum those counts across all designs.

What small change to your recursive function would give you a count instead of a boolean?

In [ ]:
def can_build(s, patterns):
    @functools.cache
    def can_build_rec(s, index):
        if index == len(s):
            return 1
        return sum(s[index:].startswith(p) and can_build_rec(s, index + len(p)) for p in patterns)
    return can_build_rec(s, 0)
L(ds).map(can_build, patterns=ps)

[2, 1, 4, 6, 0, 1, 2, 0]

In [ ]:
solve(samp)

16

In [ ]:
solve(inp)

777669668613191

# Reflection

Let's reflect - is there any part which could be improved?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Nice solution! One small thing: `s` is passed to `can_build_rec` but it never changes — it's always the same string. Since it's a cache key, you're caching more than needed.

Do you see how to remove it from the inner function's parameters?

In [ ]:
def can_build(s, patterns):
    @functools.cache
    def can_build_rec(index):
        if index == len(s):
            return 1
        return sum(s[index:].startswith(p) and can_build_rec(index + len(p)) for p in patterns)
    return can_build_rec(0)
L(ds).map(can_build, patterns=ps)

[2, 1, 4, 6, 0, 1, 2, 0]

In [ ]:
solve(inp)

777669668613191

Let's reflect - is there any part which could be improved?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good fix! Another small optimization: `s[index:].startswith(p)` creates a new string slice each time. You can avoid that with `s.startswith(p, index)` which checks starting at position `index` directly.

In [ ]:
def can_build(s, patterns):
    @functools.cache
    def can_build_rec(index):
        if index == len(s):
            return 1
        return sum(s.startswith(p, index) and can_build_rec(index + len(p)) for p in patterns)
    return can_build_rec(0)
L(ds).map(can_build, patterns=ps)

[2, 1, 4, 6, 0, 1, 2, 0]

Let's reflect - is there any part which could be improved?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Looks clean now! One minor thing: since the function returns a count rather than a boolean, the name `can_build` is a bit misleading. Something like `count_ways` might be clearer.

Otherwise, this is a solid, efficient solution. Well done!